# RNN (Réseaux de Neurones Récurrents) - Version Simplifiée

## 🎯 Objectifs de ce notebook

1. **Comprendre** ce qu'est un RNN et pourquoi on l'utilise
2. **Visualiser** comment un RNN traite les données séquentielles
3. **Implémenter** un RNN simple avec TensorFlow
4. **Appliquer** les RNN à un problème concret : l'analyse de sentiment
5. **Identifier** les limites des RNN basiques

---

## 1. 🤔 Qu'est-ce qu'un RNN ?

### Analogie simple :
Imaginez que vous lisez une phrase mot par mot. À chaque mot, vous :
- **Regardez** le mot actuel
- **Vous souvenez** de ce que vous avez lu avant
- **Formez** une nouvelle compréhension

C'est exactement ce que fait un RNN !

### Différence avec les réseaux classiques :
- **Réseau classique** : Une image → Une prédiction
- **RNN** : Une séquence (texte, audio, série temporelle) → Une ou plusieurs prédictions

### Exemples d'applications :
- 📱 **Traduction automatique** : "Hello" → "Bonjour"
- 😊 **Analyse de sentiment** : "J'adore ce film" → Positif
- 📈 **Prédiction de séries temporelles** : Prix des actions
- 🎵 **Génération de musique** : Composer de nouvelles mélodies

## 2. 📦 Installation et imports

In [1]:
# Installation des bibliothèques nécessaires
!pip install tensorflow numpy matplotlib scikit-learn

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [2]:
# Imports nécessaires
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Configuration pour l'affichage
plt.style.use('default')

print(f"✅ TensorFlow version: {tf.__version__}")
print(f"✅ Imports terminés !")

ModuleNotFoundError: No module named 'tensorflow'

## 3. 🧠 Comment fonctionne un RNN ?

### Visualisation conceptuelle

Un RNN traite une séquence étape par étape :

```
Mot 1: "J'"     → État caché h1
Mot 2: "adore"  → État caché h2 (utilise h1 + "adore")
Mot 3: "ce"     → État caché h3 (utilise h2 + "ce")
Mot 4: "film"   → État caché h4 (utilise h3 + "film")
                → Prédiction finale: POSITIF
```

### Formule mathématique (simplifiée) :
```
État caché(t) = tanh(Poids × [État précédent, Mot actuel])
```

In [ ]:
# Visualisation simple du fonctionnement d'un RNN
def visualiser_rnn():
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    
    # Positions des éléments
    positions = [1, 3, 5, 7]
    mots = ["J'", "adore", "ce", "film"]
    
    # Dessiner les cellules RNN
    for i, (pos, mot) in enumerate(zip(positions, mots)):
        # Cellule RNN
        rect = plt.Rectangle((pos-0.3, 2), 0.6, 1, 
                           facecolor='lightblue', edgecolor='darkblue', linewidth=2)
        ax.add_patch(rect)
        ax.text(pos, 2.5, 'RNN', ha='center', va='center', fontsize=12, fontweight='bold')
        
        # Mot d'entrée
        ax.text(pos, 1, mot, ha='center', va='center', fontsize=12, 
                bbox=dict(boxstyle="round,pad=0.3", facecolor='lightgreen'))
        
        # Flèche d'entrée
        ax.arrow(pos, 1.5, 0, 0.4, head_width=0.1, head_length=0.1, fc='green', ec='green')
        
        # État caché
        ax.text(pos, 3.5, f'h{i+1}', ha='center', va='center', fontsize=10,
                bbox=dict(boxstyle="round,pad=0.2", facecolor='orange'))
        
        # Flèche de sortie
        ax.arrow(pos, 3.1, 0, 0.3, head_width=0.1, head_length=0.1, fc='orange', ec='orange')
        
        # Connexion horizontale (sauf pour le dernier)
        if i < len(positions) - 1:
            ax.arrow(pos + 0.3, 2.5, 1.4, 0, head_width=0.1, head_length=0.1, 
                    fc='red', ec='red', linewidth=2)
    
    # Prédiction finale
    ax.text(8.5, 2.5, 'POSITIF', ha='center', va='center', fontsize=14, fontweight='bold',
            bbox=dict(boxstyle="round,pad=0.5", facecolor='gold'))
    ax.arrow(7.3, 2.5, 0.9, 0, head_width=0.1, head_length=0.1, fc='blue', ec='blue')
    
    # Légendes
    ax.text(4, 0.2, 'Mots d\'entrée', ha='center', fontsize=10, color='green')
    ax.text(4, 4.2, 'États cachés', ha='center', fontsize=10, color='orange')
    ax.text(4, 1.8, 'Connexions récurrentes', ha='center', fontsize=10, color='red')
    
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 5)
    ax.set_title('🧠 Comment un RNN traite la phrase "J\'adore ce film"', 
                fontsize=16, fontweight='bold', pad=20)
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()

visualiser_rnn()

## 4. 🛠️ Premier RNN avec TensorFlow

Créons notre premier RNN étape par étape :

In [ ]:
# Étape 1: Créer un RNN simple
def creer_rnn_simple():
    """
    Crée un RNN basique pour comprendre la structure
    """
    model = models.Sequential([
        # Couche RNN simple
        # - 64: nombre de neurones dans l'état caché
        # - input_shape: (longueur_sequence, dimensions_features)
        layers.SimpleRNN(64, input_shape=(10, 1)),
        
        # Couche de sortie pour classification binaire
        layers.Dense(1, activation='sigmoid')
    ])
    
    return model

# Créer le modèle
mon_premier_rnn = creer_rnn_simple()

# Voir la structure
print("📋 Structure de notre premier RNN :")
mon_premier_rnn.summary()

In [ ]:
# Étape 2: Tester avec des données factices
print("🧪 Test avec des données exemple :")

# Créer des données d'exemple
# Format: (nombre_exemples, longueur_sequence, features)
donnees_test = np.random.randn(5, 10, 1)  # 5 exemples, séquences de 10, 1 feature

print(f"📊 Forme des données d'entrée: {donnees_test.shape}")

# Faire une prédiction
predictions = mon_premier_rnn.predict(donnees_test, verbose=0)

print(f"📈 Forme des prédictions: {predictions.shape}")
print(f"📋 Premières prédictions: {predictions.flatten()[:3]}")

print("\n✅ Notre RNN fonctionne ! Il prend des séquences et produit des prédictions.")

## 5. 📝 Projet pratique : Analyse de sentiment

Maintenant, appliquons les RNN à un problème réel : déterminer si un commentaire est positif ou négatif.

### Étapes :
1. **Créer** des données d'exemple
2. **Preprocesser** le texte
3. **Construire** le modèle RNN
4. **Entraîner** le modèle
5. **Tester** sur de nouveaux exemples

In [ ]:
# Étape 1: Créer nos données d'entraînement
def creer_donnees_sentiment():
    """
    Crée un petit dataset pour l'analyse de sentiment
    """
    # Commentaires POSITIFS
    positifs = [
        "Ce film est absolument fantastique",
        "J'ai adoré cette expérience",
        "Excellent service, très satisfait",
        "Produit de qualité exceptionnelle",
        "Une expérience merveilleuse",
        "Superbe performance des acteurs",
        "Je recommande vivement",
        "Très bonne qualité",
        "Service client parfait",
        "Vraiment impressionnant"
    ]
    
    # Commentaires NÉGATIFS
    negatifs = [
        "Film très décevant et ennuyeux",
        "Très mauvaise expérience",
        "Produit de mauvaise qualité",
        "Je suis très déçu",
        "À éviter absolument",
        "Service client inexistant",
        "Qualité vraiment médiocre",
        "Perte de temps total",
        "Je regrette cet achat",
        "Vraiment catastrophique"
    ]
    
    # Combiner les données
    tous_textes = positifs + negatifs
    
    # Labels: 1 = positif, 0 = négatif
    labels = [1] * len(positifs) + [0] * len(negatifs)
    
    # Multiplier les données pour avoir plus d'exemples
    tous_textes = tous_textes * 5  # 100 exemples au total
    labels = labels * 5
    
    return tous_textes, np.array(labels)

# Créer les données
textes, sentiments = creer_donnees_sentiment()

print(f"📊 Nombre total d'exemples: {len(textes)}")
print(f"📊 Positifs: {np.sum(sentiments)}, Négatifs: {len(sentiments) - np.sum(sentiments)}")
print("\n📋 Exemples:")
for i in range(3):
    emoji = "😊" if sentiments[i] == 1 else "😞"
    print(f"  {emoji} {textes[i]}")

In [ ]:
# Étape 2: Preprocesser les textes
print("🔧 Préparation des données...")

# Configuration
MAX_MOTS = 1000  # Vocabulaire maximum
MAX_LONGUEUR = 15  # Longueur maximale des phrases

# Créer le tokenizer (convertit les mots en nombres)
tokenizer = Tokenizer(num_words=MAX_MOTS)
tokenizer.fit_on_texts(textes)

print(f"📚 Taille du vocabulaire: {len(tokenizer.word_index)}")

# Convertir les textes en séquences de nombres
sequences = tokenizer.texts_to_sequences(textes)

print(f"📋 Exemple de conversion:")
print(f"  Texte: '{textes[0]}'")
print(f"  Séquence: {sequences[0]}")

# Égaliser la longueur des séquences (padding)
X = pad_sequences(sequences, maxlen=MAX_LONGUEUR)
y = sentiments

print(f"\n📊 Forme finale des données: {X.shape}")
print(f"📊 Exemple après padding: {X[0]}")

# Diviser en train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\n✅ Données prêtes !")
print(f"   Train: {X_train.shape[0]} exemples")
print(f"   Test: {X_test.shape[0]} exemples")

In [ ]:
# Étape 3: Construire le modèle RNN pour l'analyse de sentiment
def creer_rnn_sentiment():
    """
    Crée un RNN spécialisé pour l'analyse de sentiment
    """
    model = models.Sequential([
        # 1. Couche d'embedding (convertit les nombres en vecteurs)
        layers.Embedding(
            input_dim=MAX_MOTS,      # Taille du vocabulaire
            output_dim=50,           # Dimension des vecteurs de mots
            input_length=MAX_LONGUEUR # Longueur des séquences
        ),
        
        # 2. Couche RNN
        layers.SimpleRNN(
            64,                      # 64 neurones dans l'état caché
            dropout=0.3              # Pour éviter le surapprentissage
        ),
        
        # 3. Couche de sortie (positif ou négatif)
        layers.Dense(1, activation='sigmoid')
    ])
    
    # Compiler le modèle
    model.compile(
        optimizer='adam',                    # Algorithme d'optimisation
        loss='binary_crossentropy',         # Fonction de perte pour classification binaire
        metrics=['accuracy']                # Métrique à suivre
    )
    
    return model

# Créer le modèle
modele_sentiment = creer_rnn_sentiment()

print("🏗️ Architecture du modèle RNN pour sentiment:")
modele_sentiment.summary()

print("\n📝 Explication des couches:")
print("  🔹 Embedding: Convertit les mots (nombres) en vecteurs")
print("  🔹 SimpleRNN: Traite la séquence et mémorise le contexte")
print("  🔹 Dense: Prédiction finale (0=négatif, 1=positif)")

In [ ]:
# Étape 4: Entraîner le modèle
print("🚀 Début de l'entraînement...")

# Entraîner le modèle
historique = modele_sentiment.fit(
    X_train, y_train,           # Données d'entraînement
    batch_size=16,              # Traiter 16 exemples à la fois
    epochs=10,                  # 10 passages sur toutes les données
    validation_split=0.2,       # 20% des données pour validation
    verbose=1                   # Afficher les détails
)

print("\n✅ Entraînement terminé !")

In [ ]:
# Visualiser l'entraînement
def visualiser_entrainement(historique):
    """
    Affiche l'évolution de l'accuracy et de la loss pendant l'entraînement
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Graphique de l'accuracy
    ax1.plot(historique.history['accuracy'], 'b-', label='Train', linewidth=2)
    ax1.plot(historique.history['val_accuracy'], 'r-', label='Validation', linewidth=2)
    ax1.set_title('📈 Précision du modèle', fontsize=14)
    ax1.set_xlabel('Époque')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Graphique de la loss
    ax2.plot(historique.history['loss'], 'b-', label='Train', linewidth=2)
    ax2.plot(historique.history['val_loss'], 'r-', label='Validation', linewidth=2)
    ax2.set_title('📉 Erreur du modèle', fontsize=14)
    ax2.set_xlabel('Époque')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

visualiser_entrainement(historique)

In [ ]:
# Étape 5: Tester le modèle
print("🧪 Évaluation sur les données de test:")

# Évaluer sur les données de test
test_loss, test_accuracy = modele_sentiment.evaluate(X_test, y_test, verbose=0)
print(f"📊 Précision sur le test: {test_accuracy:.1%}")

# Fonction pour prédire le sentiment d'une nouvelle phrase
def predire_sentiment(phrase):
    """
    Prédit si une phrase est positive ou négative
    """
    # Preprocesser la phrase
    sequence = tokenizer.texts_to_sequences([phrase])
    sequence_paddee = pad_sequences(sequence, maxlen=MAX_LONGUEUR)
    
    # Faire la prédiction
    score = modele_sentiment.predict(sequence_paddee, verbose=0)[0, 0]
    
    # Interpréter le résultat
    if score > 0.5:
        return "😊 POSITIF", score
    else:
        return "😞 NÉGATIF", score

# Tester sur de nouvelles phrases
nouvelles_phrases = [
    "Ce restaurant est formidable",
    "Je deteste ce produit",
    "Très bonne qualité",
    "Service décevant",
    "Pas mal mais peut mieux faire"
]

print("\n🔮 Prédictions sur de nouvelles phrases:")
print("=" * 50)

for phrase in nouvelles_phrases:
    sentiment, score = predire_sentiment(phrase)
    print(f"'{phrase}'")
    print(f"  → {sentiment} (confiance: {score:.1%})\n")

## 6. 🔍 Comprendre les limites des RNN

Les RNN basiques ont quelques problèmes importants :

In [ ]:
# Démonstration du problème de mémoire courte
def tester_memoire_rnn():
    """
    Montre comment les RNN ont du mal avec les longues séquences
    """
    # Phrase courte vs phrase longue
    phrase_courte = "Ce film est génial"
    phrase_longue = "Ce film, malgré quelques défauts mineurs dans le scénario et des moments qui traînent un peu en longueur, reste génial"
    
    print("🔬 Test de mémoire des RNN:")
    print("\n📝 Phrase courte:")
    print(f"  '{phrase_courte}'")
    sentiment, score = predire_sentiment(phrase_courte)
    print(f"  → {sentiment} (confiance: {score:.1%})")
    
    print("\n📝 Phrase longue (même sentiment final):")
    print(f"  '{phrase_longue}'")
    sentiment, score = predire_sentiment(phrase_longue)
    print(f"  → {sentiment} (confiance: {score:.1%})")
    
    print("\n💡 Observation:")
    print("   Les RNN ont plus de difficulté avec les phrases longues")
    print("   car ils 'oublient' le début de la phrase.")

tester_memoire_rnn()

In [ ]:
# Résumé des limitations
def afficher_limitations():
    """
    Montre les principales limitations des RNN
    """
    limitations = {
        "🧠 Mémoire courte": "Oublient les informations lointaines dans la séquence",
        "⏳ Lenteur d'entraînement": "Doivent traiter les mots un par un (pas de parallélisation)",
        "📉 Gradient qui disparaît": "Difficulté à apprendre sur de longues séquences",
        "🔄 Pas de vision d'ensemble": "Ne voient que ce qui précède, pas la suite"
    }
    
    solutions = {
        "🧠 Mémoire courte": "→ LSTM et GRU (avec portes de mémoire)",
        "⏳ Lenteur d'entraînement": "→ Transformers (traitement parallèle)",
        "📉 Gradient qui disparaît": "→ LSTM/GRU avec connexions résiduelles",
        "🔄 Pas de vision d'ensemble": "→ Attention mechanisms et Transformers"
    }
    
    print("⚠️ Limitations des RNN basiques:")
    print("=" * 60)
    
    for limitation, description in limitations.items():
        print(f"\n{limitation}")
        print(f"  {description}")
        if limitation in solutions:
            print(f"  {solutions[limitation]}")

afficher_limitations()

## 7. 🎯 Résumé et Points Clés

### ✅ Ce que nous avons appris :

1. **Les RNN sont parfaits pour les données séquentielles** (texte, audio, séries temporelles)
2. **Ils maintiennent un "état caché"** qui se souvient du contexte précédent
3. **TensorFlow/Keras rend l'implémentation simple** avec quelques lignes de code
4. **Les applications sont nombreuses** : sentiment, traduction, génération de texte

### ⚠️ Limitations importantes :

1. **Mémoire courte** pour les longues séquences
2. **Entraînement lent** (traitement séquentiel)
3. **Problème du gradient qui disparaît**

### 🚀 Prochaines étapes :

- **LSTM et GRU** : Versions améliorées des RNN
- **Transformers** : Architecture moderne qui résout la plupart des problèmes
- **Applications avancées** : Traduction automatique, chatbots

---

### 💡 Exercices pour aller plus loin :

1. **Modifiez le dataset** : Ajoutez vos propres phrases positives/négatives
2. **Changez l'architecture** : Testez différents nombres de neurones
3. **Essayez d'autres problèmes** : Classification de genres de films, détection de spam
4. **Comparez avec un modèle simple** : Bag of Words vs RNN

Félicitations ! 🎉 Vous maîtrisez maintenant les bases des RNN !